# Evaluating the Exact Hessian of Deep Neural Networks: Autograd v.s. Tensorflow

This is a tutorial style evaluation of the Hessian matrix for multi layer Neural Networks. We do this both in autograd and in Tensorflow. Evaluation using the former has been partially ducumented elsewhere, and seems more established, while I did not find a correct implementation of the latter. Indeed, you will find that the `tf.hessian()` function is not evaluating the exact hessian.   

In order to simplify things and being able to directly inspect the results, we consider a simple 2 layer linear network with Nfeat =3 input features, Nh = 2 hidden units and Nlab=1 output. Let us start with Autograd   

![alt text](FF1.pdf)

The linear network is defined by two simple linear operations. The linear output of the two layers is denotes as 

$$h^{[1]}_i = \sum_j w^{[1]}_{ij} \, X_j + b^{[1]}_i $$ 

and 

$$h^{[2]}_i = \sum_j w^{[2]}_{ij} \, h^{[1]}_j + b^{[2]}_i, $$

where the superscipt denotes the layer and the lower indices denote the elements of the vector/matrix. Finally, we need to define the Loss function. Here we use the minimum squared error loss, but you can use your favourite one. 

$$ L = \frac{1}{2 m } \sum_{\mu} (\hat{y}^{\mu} - y^{\mu} )^2 $$ 
where in this simple case $\hat{y} = h_2$

Note that in this tutorial we are not going to implement any optimization routine, but merely show how to evaluate the Hessian for a fixed value of the parameters. You can easily implement the code here to work with your gradient descent algo.

**Hessian** In general, defining the vector $\theta_{\alpha} = (\mathbf{w}_1, \mathbf{b}_1, \mathbf{w}_2, \mathbf{b}_2)$, the Hessian matrix is given by 


$$
H_{\alpha, \beta} = \frac{\partial^2 L}{\partial \theta_{\alpha} \, \partial \theta_{\beta}} =
\begin{pmatrix}
\frac{\partial^2 L}{\partial w^{[1]}_{ij} \, \partial w^{[1]}_{lm}} & \frac{\partial^2 L}{\partial w^{[1]}_{ij} \, \partial b^{[1]}_{lm}} & \frac{\partial^2 L}{\partial w^{[1]}_{ij} \, \partial w^{[2]}_{lm}} & \frac{\partial^2 L}{\partial w^{[1]}_{ij} \, \partial b^{[2]}_{l}}\\
\frac{\partial^2 L}{\partial b^{[1]}_{i} \, \partial w^{[1]}_{lm}} &  \frac{\partial^2 L}{\partial b^{[1]}_{i} \, \partial b^{[1]}_{j}} &  \frac{\partial^2 L}{\partial b^{[1]}_{i} \, \partial w^{[2]}_{lm}} &  \frac{\partial^2 L}{\partial b^{[1]}_{i} \, \partial b^{[2]}_{j}}  \\
\frac{\partial^2 L}{\partial w^{[2]}_{ij} \, \partial w^{[1]}_{lm}} & \frac{\partial^2 L}{\partial w^{[2]}_{ij} \, \partial b^{[1]}_{l}} & \frac{\partial^2 L}{\partial w^{[2]}_{ij} \, \partial w^{[2]}_{lm}} &  \frac{\partial^2 L}{\partial w^{[2]}_{ij} \, \partial b^{[2]}_{l}} \\ 
\frac{\partial^2 L}{\partial b^{[2]}_{i} \, \partial w^{[1]}_{lm}} & \frac{\partial^2 L}{\partial b^{[2]}_{i} \, \partial b^{[1]}_{j}} & \frac{\partial^2 L}{\partial b^{[2]}_{i} \, \partial w^{[2]}_{lm}} &  \frac{\partial^2 L}{\partial b^{[2]}_{i} \, \partial b^{[2]}_{j}}
\end{pmatrix} = 
\begin{pmatrix} 
H_{11} & H_{12} \\
H_{21} & H_{22}
\end{pmatrix}
$$

where in the last identity we have used a compact, block notation in layer space. Note that each element appearing in the big matrix is itself a matrix. The off-diagonal blocks are important if oyu want to capture correlations between different layers, that are crucial to the deep learning framework. With respect to this, I have seen in the literature something called "diagonal approximation", however to me it looks like a rather bold step. Clarly, if the off-diagonal elements are small, you can obtain an approximate block diagonal form in which the Hessian factorizes. Reaining only the block diagonal part means neglecting correlations between different layers, i.e. saying that those are not important! But these are at the core of the Deep Learning framework ... a perturbative expansion it is certainly more appropriate.  


As we want to compare results with the tensor flow implementation, we need to make sure that the weights are initialized in the same way in both implementations. Below we first define the input matrix and initialize the weights and biases by sampling from a normal distirbution. 


In [2]:
import autograd.numpy as np #autograd version of numpy

x = np.array([[0.52, 1.12,  0.77],
             [0.88, -1.08, 0.15],
             [0.52, 0.06, -1.30],
             [0.74, -2.49, 1.39]])

m, Nfeat = x.shape # m = number of training sampples, f = number of features

#Define the ouput
y= x[:,0]**2-3*x[:,1]*x[:,2]
y = np.reshape(y, (m,1))

_, Nlab = y.shape #number of units in the ouput layer

Nh= 2  #number of units in the hidden layer

#initialize parameters to random value

W1 = np.random.randn(2,3)
B1 = np.random.randn(2,1)

W2 = np.random.randn(1,2)
B2 = np.random.randn(1,1)

# Autograd

[Autograd](https://github.com/WessZumino/autograd) is a nice library that allows you to perform automatic differentiation on native Python and Numpy code. We first import the relevant libraries  

In [3]:
from flatten import*
from autograd import grad, hessian

Define first some useful helper functions to evaluate the linear ouput of the network and the loss function.

In [10]:
#Linear ouputs
def nn_outs(params, inputs):
    for W, b in params:
        outputs = np.dot(W, inputs) + b
        inputs = outputs
    return outputs

#Loss function for the regression problem
def loss(weights, input, target):

        h2 = nn_outs(weights, input)

        return np.mean(np.square(h2-target))/2

### Implementation

This is quite straightforward in Autograd, where we will use the native Hessian function


In [11]:
#initialize paramters: returns 4 params
init_params = [(W1, B1), (W2, B2)]

#Initial value of the objective funciton
loss(init_params, x.T, y.T)

#evaluate the gradients
grads_ag = grad(objective)(init_params, x.T, y.T)

#returns a list with four arrays containing dw1, db1, dw2, db2
print('Gradients:',grads_ag)

#flatten the objective + initialization paramters. This step is needed to pass the input as a one dimensional 
#array to the evaluate Hessian function.

flat_f, unflatten, flat_params = flatten_func(objective, init_params)

print('flattened parameters:',flat_params)

#evaluate the Hessian
flat_hess = hessian(flat_f) #initialize the Hessian

hess_ag = flat_hess(flat_params, x.T, y.T) #evaluates the Hessian on the input/ouput data

#we can now check that this has the expected dimensions: (2*3)+(2*1)+(1*2)+1 = 11 --> hess= (11,11)
print('dimension of the Hessian matrix:',hess_ag[0].shape)

Gradients: [(array([[-0.20980657, -5.42853043,  1.58130118],
       [ 0.20253104,  5.24028375, -1.52646595]]), array([[-0.75951313],
       [ 0.73317528]])), (array([[-4.41988815, -3.68119392]]), array([[0.76095164]]))]
flattened parameters: [-0.38575379 -0.62680877  0.19602057  0.34699285 -0.51387488  1.07746459
 -0.81365418  0.98266259 -0.99810959  0.96349786  0.98364881]
dimension of the Hessian matrix: (11, 11)


# Tensorflow

We are now going to implement the same calculation using tensorflow (TF). I assume you have a basic knowlede if it, but I will try to be comprehensive anyway

In [8]:
import scipy as sp

import tensorflow as tf
from tensorflow.python.framework import ops


/Applications/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Initialize the Network

Unlike in the previous example, in TF one works with symbolic tensors, values are passed to them at running time. This is like when you solve a physics/math problem with pen and paper! Also in this case it is convenient to carry on calculations with symbolic variables until the very end, where you are going to input the actual values of these variables.

These symbolic tensors are called place holders in TF for obvious reasons. We define one for the input and one for the output. Note that "None" means that we do not have to specify the number of training examples in the dataset for the moment (in our case this is trivial, m=1).   

In [13]:
X = tf.placeholder(shape= [Nfeat, None], dtype= "float64", name= 'X' )
Y = tf.placeholder(shape= [Nlab, None],  dtype= "float64", name= 'Y' )

**Initialize the variables**. For each layer we need to specify the weights (i.e the connections) and the biases. So for this simple case we have a total of 4 parameters. As we want to compare with the Autograd result, we initialize the weights in the same way 

In [14]:
#Define first the initializers: 

w1_init = tf.constant_initializer(W1)
b1_init = tf.constant_initializer(B1)

w2_init= tf.constant_initializer(W2)
b2_init= tf.constant_initializer(B2)

#Use them to initialized the weights and bias:
w1= tf.get_variable('w1', [Nh, Nfeat], dtype= 'float64' ,initializer = w1_init )
b1= tf.get_variable('b1', [Nh, 1], dtype= 'float64', initializer = b1_init)

w2= tf.get_variable('w2', [Nlab, Nh], dtype= 'float64' ,initializer = w2_init )
b2= tf.get_variable('b2', [Nlab, 1], dtype= 'float64', initializer = b2_init )


**Forward pass and Loss**

In [15]:
#Define the FW pass to the two units
h1 = tf.matmul(w1, X)+b1
h2 = tf.matmul(w2, h1)+b2

#define the loss
L = tf.reduce_mean(tf.squared_difference(h2, Y))/2


Before moving on, we get a list of all the trainable variables 

In [17]:
tvars = tf.trainable_variables()
print('Trainable parametrs:',tvars)

Trainable parametrs: [<tf.Variable 'w1:0' shape=(2, 3) dtype=float64_ref>, <tf.Variable 'b1:0' shape=(2, 1) dtype=float64_ref>, <tf.Variable 'w2:0' shape=(1, 2) dtype=float64_ref>, <tf.Variable 'b2:0' shape=(1, 1) dtype=float64_ref>]


## Operations on the Network : Gradients and Hessian matrix 

Here we define all the operations we want to perform. The **gradient** of the Loss function with respect to the training variables is easily obtained using the `tf.gradients()` function. Note: do not use $[0]$ as shown in some other tutorials, otherwise you get only dw1!

In [18]:
dL = tf.gradients(L, tvars )
print('Gradients:',dL)

Gradients: [<tf.Tensor 'gradients/MatMul_grad/MatMul:0' shape=(2, 3) dtype=float64>, <tf.Tensor 'gradients/add_grad/Reshape_1:0' shape=(2, 1) dtype=float64>, <tf.Tensor 'gradients/MatMul_1_grad/MatMul:0' shape=(1, 2) dtype=float64>, <tf.Tensor 'gradients/add_1_grad/Reshape_1:0' shape=(1, 1) dtype=float64>]


As you can see from the ouput, dL is a list containing ($ \partial_{w1} L,\, \partial_{b1} L,\, \partial_{w2} L,\, \partial_{b2} L $ ). So the first index $dL[i]$ selects one of the 4 gradients. The other indices give access to the elements of the gradients: e.g. $d_{w1}$ is a (2,3) matrix, so $dL[0][i][j]$ gives access to the $i,j$ elements of the matrix.   

**Hessian**: This part is a bit more tedious as the `tf.gradients()` function does not differentiate tensors of rank>1. A simple way of avoiding this problem is to pass slices of the orginal gradients (computed before) such that we pass each element one by one. In order to do that we first create a flattenning fucntion, just like the one that was provided in the Autograd implementation:



In [19]:
#flattening function

def flatten(tensor):

    '''
    Flattening function:

    input: a tensor list
    returns: a rank one tensor
    '''

    s= len(tensor) #number of tensors in the list

    for i in range(s):

        dl = tensor[i] #take one element of the list
        d1, d2 = dl.get_shape() #Obtain tensor dimensions

        fl = tf.reshape(dl,[-1, d1*d2]) #reshape the tensor to a (1, d1*d2) tensor

        #concatenate over all the elemets in the list
        if i==0: flattened = fl # the first time
        else: flattened = tf.concat([flattened, fl], axis=1)

    return flattened


Likewise, we define another function to evaluate the Hessian

In [21]:
#Hessian function

def hessian(grads, par):

    '''
    Evaluates the Hessian matrix

    Inputs:
    grads --- the evaluated gradeints of the cost function

    Returns:
    hessian matrix: a (dim,dim) matrix of second derivatives, where 'dim' is the dimension of
    the flattened gradient tensor.
    '''

    flat_grads = flatten(grads)[0] #flat gradients

    dim = flat_grads.get_shape()[0] #get the dimensions of the flattened tensor

    hess = [] #list

    for i in range (dim):

        dg_i = tf.gradients(flat_grads[i], par) #for each element of grads evaluate the gradients
        dg_i_flat = flatten(dg_i) #flatten the resulting hessian onto a 1 d array
        hess.append(dg_i_flat) #store row by row

    return tf.reshape(hess,[dim, dim]) #returns the reshaped matrix


We now apply the flattening function to the parameters and the gradients and use the result as an input to the Hessian function 

In [22]:
flat_grads = flatten(dL)[0] #flattened gradients 
flat_par= flatten(tvars)[0] #flattened parameters

#Evaluate the symbolic Hessian 
hess = hessian(dL, tvars) #evaluates the hessian


At this point we open a TF session in order to evaluate the symbolic Hessian on the data

In [23]:
init = tf.global_variables_initializer() #initialize the computational graph

with tf.Session() as sess:

    sess.run(init)
    grads_tf, grads_flat, flat_params_tf, hess_tf = sess.run([dL, flat_grads, flat_par, hess], feed_dict = {X: x.T, Y: y.T} )


## Comparing the results from Autograd and Tensorflow

we can now print the result of the different operations we have perfomed so far, both in Autograd and Tensorflow 

In [32]:
#This is just to make sure we are using the same parameters 
print('Autograd parameters',flat_params)
print('Tensorflow paramters', flat_params_tf )


Autograd parameters [-0.38575379 -0.62680877  0.19602057  0.34699285 -0.51387488  1.07746459
 -0.81365418  0.98266259 -0.99810959  0.96349786  0.98364881]
Tensorflow paramters [-0.38575379 -0.62680877  0.19602057  0.34699285 -0.51387488  1.07746459
 -0.81365418  0.98266259 -0.99810959  0.96349786  0.98364881]


In [36]:
#Evaluated Hessian matrix

print( 'Hessian from Autograd:', hess_ag[0][10])
print('Hessian from Tensorflow:', hess_tf[10] )


Hessian from Autograd: [-0.66374288  0.59637048 -0.25202267  0.64072608 -0.57568997  0.24328321
 -0.99810959  0.96349786 -0.64616702  1.79251289  1.        ]
Hessian from Tensorflow: [-0.66374288  0.59637048 -0.25202267  0.64072608 -0.57568997  0.24328321
 -0.99810959  0.96349786 -0.64616702  1.79251289  1.        ]
